In [4]:
import os
import json
from pathlib import Path
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
from loguru import logger

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [5]:
from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [6]:
logger.remove()

In [7]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
curr_dir = msmri_home / "analysis" / "choroid_pineal2_pituitary_crosstrain_t1"
training_work_dirs = msmri_home / "training_work_dirs"
modalities = ["t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]
label_suffix = ["SRS_T1", "CH", "SRS", "ED", "DT"]

### First Set

In [8]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal2_pituitary_T1-1"
work_dir = training_work_dirs / work_dir_name

In [9]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset1.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, label_suffix)
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 254.63it/s]


In [10]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [11]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [12]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-03-06 18:18:58,525 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1
2025-03-06 18:18:58,527 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/datalist.json.
2025-03-06 18:18:58,527 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/datalist.json.
2025-03-06 18:18:58,541 - INFO - Using user defined command running prefix , will override other settings


In [13]:
runner.run()

2025-03-06 18:18:58,545 - INFO - Running data analysis...
2025-03-06 18:18:58,545 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [01:05<00:00,  2.19s/it]

2025-03-06 18:20:04,423 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-03-06 18:20:04,424 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/datastats.yaml.
2025-03-06 18:20:04,429 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/datastats_by_case.yaml, this may take a while.
2025-03-06 18:20:04,515 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 526kB/s]                              

2025-03-06 18:20:04,719 - INFO - Downloaded: /tmp/tmp6v9to50r/algo_templates.tar.gz
2025-03-06 18:20:04,719 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmp6v9to50r/algo_templates.tar.gz.
2025-03-06 18:20:04,719 - INFO - Writing into directory: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1.
2025-03-06 18:20:04,767 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_0
2025-03-06 18:20:04,794 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_1
2025-03-06 18:20:04,819 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_2
2025-03-06 18:20:04,844 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_3
2025-03-06 18:20:04,870 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunet

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-03-06 23:50:29,447 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-03-06 23:50:29,447 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/03/06 23:50:29 INFO mlflow.

2025-03-07 01:30:34,437 - INFO - Ensembling using single GPU!
2025-03-07 01:30:34,438 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/ensemble_output will be used to save ensemble predictions.
2025-03-07 01:30:34,438 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/ensemble_output is created to save ensemble predictions
2025-03-07 01:30:34,463 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-03-07 01:30:34,463 - INFO - swinunetr_0
2025-03-07 01:30:34,463 - INFO - swinunetr_1
2025-03-07 01:30:34,464 - INFO - swinunetr_2
2025-03-07 01:30:34,464 - INFO - swinunetr_3
2025-03-07 01:30:34,464 - INFO - swinunetr_4
2025-03-07 01:30:34,464 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-1/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Second Set

In [14]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal2_pituitary_T1-2"
work_dir = training_work_dirs / work_dir_name

In [15]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset2.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, label_suffix)
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 251.90it/s]


In [16]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [17]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [18]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-03-07 01:40:15,392 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2
2025-03-07 01:40:15,393 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/datalist.json.
2025-03-07 01:40:15,393 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/datalist.json.
2025-03-07 01:40:15,393 - INFO - Using user defined command running prefix , will override other settings


In [19]:
runner.run()

2025-03-07 01:40:15,406 - INFO - Running data analysis...


2025-03-07 01:40:15,407 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [01:16<00:00,  2.56s/it]

2025-03-07 01:41:32,321 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-03-07 01:41:32,321 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/datastats.yaml.
2025-03-07 01:41:32,327 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/datastats_by_case.yaml, this may take a while.
2025-03-07 01:41:32,447 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 534kB/s]                              

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-03-07 08:39:02,458 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-03-07 08:39:02,458 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/03/07 08:39:02 INFO mlflow.

2025-03-07 09:56:28,258 - INFO - Ensembling using single GPU!
2025-03-07 09:56:28,259 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/ensemble_output will be used to save ensemble predictions.
2025-03-07 09:56:28,259 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/ensemble_output is created to save ensemble predictions
2025-03-07 09:56:28,283 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-03-07 09:56:28,284 - INFO - swinunetr_0
2025-03-07 09:56:28,284 - INFO - swinunetr_1
2025-03-07 09:56:28,284 - INFO - swinunetr_2
2025-03-07 09:56:28,284 - INFO - swinunetr_3
2025-03-07 09:56:28,284 - INFO - swinunetr_4
2025-03-07 09:56:28,285 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-2/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Third Set

In [20]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal2_pituitary_T1-3"
work_dir = training_work_dirs / work_dir_name

In [21]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset3.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, label_suffix)
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 250.74it/s]


In [22]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [23]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [24]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-03-07 10:05:39,575 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3
2025-03-07 10:05:39,576 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/datalist.json.
2025-03-07 10:05:39,576 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/datalist.json.
2025-03-07 10:05:39,577 - INFO - Using user defined command running prefix , will override other settings


In [25]:
runner.run()

2025-03-07 10:05:39,590 - INFO - Running data analysis...


2025-03-07 10:05:39,591 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [01:04<00:00,  2.14s/it]

2025-03-07 10:06:43,799 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-03-07 10:06:43,800 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/datastats.yaml.
2025-03-07 10:06:43,805 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/datastats_by_case.yaml, this may take a while.
2025-03-07 10:06:43,927 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 507kB/s]                              

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-03-07 15:31:43,063 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-03-07 15:31:43,063 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/03/07 15:31:43 INFO mlflow.

2025-03-07 16:51:46,971 - INFO - Ensembling using single GPU!
2025-03-07 16:51:46,971 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/ensemble_output will be used to save ensemble predictions.
2025-03-07 16:51:46,972 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/ensemble_output is created to save ensemble predictions
2025-03-07 16:51:46,997 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-03-07 16:51:46,997 - INFO - swinunetr_0
2025-03-07 16:51:46,998 - INFO - swinunetr_1
2025-03-07 16:51:46,998 - INFO - swinunetr_2
2025-03-07 16:51:46,998 - INFO - swinunetr_3
2025-03-07 16:51:46,998 - INFO - swinunetr_4
2025-03-07 16:51:46,999 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Fourth Set

In [26]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal2_pituitary_T1-4"
work_dir = training_work_dirs / work_dir_name

In [27]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset4.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, label_suffix)
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 251.39it/s]


In [28]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [29]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [30]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-03-07 17:01:55,741 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4
2025-03-07 17:01:55,742 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/datalist.json.
2025-03-07 17:01:55,742 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/datalist.json.
2025-03-07 17:01:55,742 - INFO - Using user defined command running prefix , will override other settings


In [31]:
runner.run()

2025-03-07 17:01:55,755 - INFO - Running data analysis...


2025-03-07 17:01:55,756 - INFO - Found 1 GPUs for data analyzing!


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [01:05<00:00,  2.18s/it]

2025-03-07 17:03:01,325 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-03-07 17:03:01,326 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/datastats.yaml.
2025-03-07 17:03:01,331 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/datastats_by_case.yaml, this may take a while.
2025-03-07 17:03:01,454 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-03-07 22:06:38,133 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-03-07 22:06:38,133 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/03/07 22:06:38 INFO mlflow.

2025-03-07 23:26:36,462 - INFO - Ensembling using single GPU!
2025-03-07 23:26:36,462 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/ensemble_output will be used to save ensemble predictions.
2025-03-07 23:26:36,463 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/ensemble_output is created to save ensemble predictions
2025-03-07 23:26:36,486 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-03-07 23:26:36,486 - INFO - swinunetr_0
2025-03-07 23:26:36,487 - INFO - swinunetr_1
2025-03-07 23:26:36,487 - INFO - swinunetr_2
2025-03-07 23:26:36,487 - INFO - swinunetr_3
2025-03-07 23:26:36,487 - INFO - swinunetr_4
2025-03-07 23:26:36,487 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal2_pituitary_T1-4/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Inference

In [32]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

1

In [33]:
#! Set these variables
work_dir_name = "choroid_pineal2_pituitary_T1-1"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal2_pituitary_T1-1_pred"
task_name = "infer_choroid_pineal2_pituitary"
modalities = ["t1"]

In [34]:
projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = drive_root / "srs-9" / "3Tpioneer_bids_predictions"

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


In [35]:
os.rename(work_dir / "datalist.json", work_dir / "training-datalist.json")

if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

In [36]:
from mri_data.file_manager import DataSet

def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

In [37]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)
dataset_train.dataroot = dataroot

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

In [38]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(modalities)
dataset_proc.dataset.sort(key=lambda s: s.subid)

100%|██████████| 535/535 [00:01<00:00, 284.82it/s]


Save the datalist and task files


In [39]:
images = []
for scan in dataset_proc.dataset:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

In [40]:
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

In [41]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [42]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/535 [00:00<?, ?it/s]

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
You are using `torch.load` with `wei

2025-03-07 23:36:59,553 INFO image_writer.py:197 - writing: /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   0%|          | 1/535 [00:54<8:03:02, 54.27s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is dep

2025-03-07 23:37:56,094 INFO image_writer.py:197 - writing: /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


...

Ensembling (rank 0)...: 100%|█████████▉| 533/535 [9:02:38<02:02, 61.37s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2025-03-08 08:39:32,541 INFO image_writer.py:197 - writing: /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms3024/ses-20161105/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


Ensembling (rank 0)...: 100%|█████████▉| 534/535 [9:03:27<00:57, 57.60s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2025-03-08 08:40:22,305 INFO image_writer.py:197 - writing: /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms3028/ses-20210301/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 535/535 [9:04:16<00:00, 61.04s/it]
